In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3
import requests
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import mean_squared_error
from surprise import Dataset, Reader
from surprise import KNNBasic
from surprise import accuracy
from surprise.model_selection import GridSearchCV
from surprise.model_selection import cross_validate
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense
from tensorflow import keras
from tensorflow.keras import layers
import random

In [ ]:
user_activity_log = pd.read_csv('user_activity.log')

In [ ]:
conn = sqlite3.connect('movies.db')
query = "SELECT * FROM movies"
movies_data = pd.read_sql_query(query, conn)

In [ ]:
api_url = "https://api.example.com/data"
response = requests.get(api_url)

if response.status_code == 200:
    data = response.json()
    # Обработка данных
else:
    print("Ошибка при запросе данных.")

In [ ]:
# Создание SQLite базы данных и сохранения данных
conn = sqlite3.connect('recommendation_data.db')
data.to_sql('user_activity', conn, if_exists='replace')

In [ ]:
data = data.drop_duplicates()

In [ ]:
scaler = MinMaxScaler()
data['rating'] = scaler.fit_transform(data['rating'].values.reshape(-1, 1))

In [ ]:
# Извлечение признаков
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(data['description'])

In [ ]:
ratings = np.random.randint(1, 6, 1000)

In [ ]:
plt.hist(ratings, bins=5, alpha=0.75, rwidth=0.85, color='b')
plt.xlabel('Рейтинг')
plt.ylabel('Количество оценок')
plt.title('Распределение рейтингов пользователей')
plt.show()

In [ ]:
data = {
    'Пользователь A': [4, 3, 5, 2, 1],
    'Пользователь B': [5, 4, 4, 1, 2]
}
df = pd.DataFrame(data)

In [ ]:
correlation_matrix = df.corr()

print(correlation_matrix)

In [ ]:
# Создаем датасет с временем начала сеансов просмотра фильмов
data = {
    'Время начала': ['2023-09-13 18:30:00', '2023-09-13 19:45:00', '2023-09-14 14:15:00'],
    'Пользователь': ['Пользователь A', 'Пользователь B', 'Пользователь A']
}
df = pd.DataFrame(data)

In [ ]:
# Преобразуем столбец времени в формат datetime
df['Время начала'] = pd.to_datetime(df['Время начала'])

In [ ]:
# Группируем данные по пользователям и анализируем среднее время между сеансами
user_sessions = df.groupby('Пользователь')['Время начала'].diff().mean()
print(user_sessions)

In [ ]:
# Создаем случайный датасет оценок фильмов
ratings = np.random.randint(1, 6, (100, 5))

In [ ]:
# Применяем метод K-средних для кластеризации пользователей
kmeans = KMeans(n_clusters=3)
kmeans.fit(ratings)

In [ ]:
# Выводим центры кластеров
print(kmeans.cluster_centers_)

In [ ]:
# Создаем датасет с жанрами фильмов
data = {
    'Фильм': ['Фильм 1', 'Фильм 2', 'Фильм 3'],
    'Жанры': ['комедия', 'боевик', 'драма']
}
df = pd.DataFrame(data)

In [ ]:
# Преобразуем жанры в числовые признаки с помощью TF-IDF
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['Жанры'])

In [ ]:
# Применяем метод K-средних для кластеризации фильмов по жанрам
kmeans = KMeans(n_clusters=2)
kmeans.fit(tfidf_matrix.toarray())

In [ ]:
# Выводим принадлежность фильмов к кластерам
print(kmeans.labels_)

In [ ]:
# Создаем датасет с историей оценок пользователей
data = {
    'Пользователь': ['Пользователь A', 'Пользователь B', 'Пользователь C'],
    'Фильм 1': [4, 2, 5],
    'Фильм 2': [3, 4, 1],
    'Фильм 3': [5, 2, 3]
}
df = pd.DataFrame(data)

In [ ]:
# Масштабируем данные
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df.iloc[:, 1:])

In [ ]:
# Применяем метод K-средних для кластеризации пользователей
kmeans = KMeans(n_clusters=2)
kmeans.fit(scaled_data)

In [ ]:
# Выводим принадлежность пользователей к кластерам
print(kmeans.labels_)

In [ ]:
# Создаем искусственный временной ряд оценок фильмов
date_rng = pd.date_range(start='2023-01-01', end='2023-12-31', freq='D')
user_ratings = np.random.randint(1, 6, size=(len(date_rng)))

In [ ]:
# Создаем DataFrame
df = pd.DataFrame({'Date': date_rng, 'User_Rating': user_ratings})
df.set_index('Date', inplace=True)

In [ ]:
# Визуализируем временной ряд
plt.figure(figsize=(12, 6))
plt.plot(df.index, df['User_Rating'], marker='o')
plt.title('Временной ряд оценок фильмов')
plt.xlabel('Дата')
plt.ylabel('Оценка пользователя')
plt.grid(True)
plt.show()

In [ ]:
# Сглаживание скользящим средним для выявления сезонных паттернов
window_size = 7  # размер окна
df['Smoothed_Rating'] = df['User_Rating'].rolling(window=window_size).mean()

In [ ]:
# Визуализируем сглаженный временной ряд
plt.figure(figsize=(12, 6))
plt.plot(df.index, df['Smoothed_Rating'], label=f'Smoothed (Window Size={window_size})', color='red')
plt.title('Сглаженный временной ряд оценок фильмов')
plt.xlabel('Дата')
plt.ylabel('Сглаженная оценка')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
# Создаем датасет
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['user_id', 'movie_id', 'rating']], reader)

In [ ]:
# Используем алгоритм коллаборативной фильтрации
sim_options = {'name': 'cosine', 'user_based': False}
algo = KNNBasic(sim_options=sim_options)

In [ ]:
# Оцениваем модель с помощью кросс-валидации
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

In [ ]:
def content_based_recommendations(user_profile, movies, top_n=10):
    # Вычисляем схожесть между профилем пользователя и фильмами
    similarities = cosine_similarity(user_profile, movies)
    
    # Получаем топ N рекомендаций
    top_indices = similarities.argsort()[-top_n:][::-1]
    recommended_movies = [movies[i] for i in top_indices]
    return recommended_movies

In [ ]:
# Инициализация модели ALS
als = ALS(maxIter=5, regParam=0.01, userCol="user_id", itemCol="movie_id", ratingCol="rating")

In [ ]:
# Обучение модели
model = als.fit(train_data)

In [ ]:
# Предсказание рейтингов для тестовых данных
predictions = model.transform(test_data)

In [ ]:
# Оценка качества модели
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
rmse = evaluator.evaluate(predictions)

In [ ]:
# Создание модели
model = Sequential()
model.add(Embedding(input_dim=num_users, output_dim=50, input_length=1))
model.add(Flatten())
model.add(Dense(1, activation='linear'))

In [ ]:
# Компиляция модели
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
# Обучение модели
model.fit([user_ids, movie_ids], ratings, epochs=5, batch_size=64)

In [ ]:
# Создание сетки гиперпараметров
param_grid = {'n_epochs': [5, 10, 15], 'lr_all': [0.002, 0.005, 0.01], 'reg_all': [0.4, 0.6, 0.8]}

In [ ]:
# Подбор лучших гиперпараметров
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

In [ ]:
# Получение лучших параметров и оценка качества
best_params = gs.best_params['rmse']
best_rmse = gs.best_score['rmse']

In [ ]:
# Создание профиля пользователя на основе истории просмотров и оценок
def user_profile(user_id, user_history):
    user_profile = {}
    for movie_id, rating in user_history:
        user_profile[movie_id] = rating
    return user_profile

In [ ]:
# Генерация персонализированных рекомендаций для пользователя
def personalized_recommendations(user_id, user_history, all_movies):
    user_profile_data = user_profile(user_id, user_history)
    recommended_movies = []

    for movie_id in all_movies:
        if movie_id not in user_profile_data:
            recommended_movies.append(movie_id)
    
    recommended_movies.sort(key=lambda movie_id: user_profile_data.get(movie_id, 0), reverse=True)
    return recommended_movies

In [ ]:
# Создание модели RankNet
def create_ranknet_model(input_dim):
    model = keras.Sequential()
    model.add(layers.Dense(64, activation='relu', input_dim=input_dim))
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(1, activation='linear'))
    return model

In [ ]:
# Обучение модели RankNet
def train_ranknet_model(model, X, y, epochs=10):
    model.compile(loss='mse', optimizer='adam')
    model.fit(X, y, epochs=epochs, batch_size=32)

In [ ]:
# Обновление рекомендательной модели с учетом новой оценки пользователя
def update_recommendation_model(model, user_id, movie_id, new_rating):
    # Получение существующих оценок пользователя
    user_history = get_user_history(user_id)

    # Добавление новой оценки к истории пользователя
    user_history.append((movie_id, new_rating))

    # Пересчитывание профиля пользователя и персонализированных рекомендаций
    user_profile_data = user_profile(user_id, user_history)
    personalized_recommendations = personalized_recommendations(user_id, user_history, all_movies)

    # Обновление модели с новой информацией о пользователе
    model.update_with_user_data(user_id, user_profile_data, personalized_recommendations)

In [ ]:
# вычисления RMSE
actual_ratings = [4, 3, 5, 2, 1]
predicted_ratings = [3.8, 2.9, 4.9, 2.2, 1.2]
rmse = np.sqrt(mean_squared_error(actual_ratings, predicted_ratings))

In [1]:
# вычисления Precision и Recall
relevant_items = [1, 2, 3, 4, 5]
recommended_items = [1, 3, 6, 7, 8]
precision = len(set(relevant_items) & set(recommended_items)) / len(recommended_items)
recall = len(set(relevant_items) & set(recommended_items)) / len(relevant_items)

In [ ]:
# вычисления MAP
def average_precision(relevant_items, recommended_items):
    # Рассчитываем Average Precision для одного пользователя
    precision_at_k = []
    for k in range(1, len(recommended_items) + 1):
        if recommended_items[k - 1] in relevant_items:
            precision_at_k.append(len(set(relevant_items) & set(recommended_items[:k])) / k)
    return sum(precision_at_k) / len(relevant_items)

In [ ]:
# Среднее значение Average Precision для всех пользователей
average_precisions = [average_precision(user['relevant_items'], user['recommended_items']) for user in users]
map_score = sum(average_precisions) / len(users)

In [ ]:
# Разделение пользователей на группы A и B
users = get_all_users()
group_a_users = random.sample(users, len(users) // 2)
group_b_users = list(set(users) - set(group_a_users))

In [ ]:
def update_recommendation_model(model, user_id, movie_id, new_rating):
    # Получение существующих оценок пользователя
    user_history = get_user_history(user_id)

    # Добавление новой оценки к истории пользователя
    user_history.append((movie_id, new_rating))

    # Пересчитывание профиля пользователя и персонализированных рекомендаций
    user_profile_data = user_profile(user_id, user_history)
    personalized_recommendations = personalized_recommendations(user_id, user_history, all_movies)

    # Обновление модели с новой информацией о пользователе
    model.update_with_user_data(user_id, user_profile_data, personalized_recommendations)